## Install Libraries

In [4]:
!pip install sentence_transformers pypdf faiss-gpu
!pip install langchain langchain-openai
!pip install pypdf
!pip install sentence-transformers
!pip install langchain-together
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 2.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manyli

## Import Libraries

In [5]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_together import Together
from langchain_together.embeddings import TogetherEmbeddings

## Load PDF

In [6]:
loader = PyPDFLoader("") ## give path of desired pdf (only pdf)
documents = loader.load()

## Split Document

In [7]:
text = RecursiveCharacterTextSplitter().split_documents(documents)

## Load Embedding Model

In [8]:
embeddings = TogetherEmbeddings(
                                  model="togethercomputer/m2-bert-80M-8k-retrieval" , ## you can change the embedding model
                                  together_api_key=""  ## write your together api key
                                )

## Create and store DB

In [9]:
# Create a vectorstore
vectorstore = FAISS.from_documents(text, embeddings)

# Save the documents and embeddings
vectorstore.save_local("vectorstore.db")

## Retrieve DB

In [10]:
# Create retriever
retriever = vectorstore.as_retriever()

## Load LLM

In [11]:
# from langchain_together import Together
def load_model(model_name="mistralai/Mixtral-8x7B-Instruct-v0.1"):
    llm = Together(
        model=model_name,
        temperature=0.7,
        max_tokens=128,
        top_k=1,
        together_api_key="" ## write your together api key
    )
    return llm

llm = load_model()

## System Prompt

In [12]:
# Define prompt template
template = """
You are an assistant for question-answering tasks.
Use the provided context only to answer the following question:

<context>
{context}
</context>

Question: {input}
"""

# Create a prompt template
prompt = ChatPromptTemplate.from_template(template)



## Create Chain

In [13]:
# Create a chain
doc_chain = create_stuff_documents_chain(llm, prompt)
chain = create_retrieval_chain(retriever, doc_chain)

## User Query

In [14]:
# User query
response = chain.invoke({"input": "Tell me about whole pdf."})



## Print Answer

In [15]:
# Get the Answer only
response['answer']

'\nAnswer: The provided context appears to be a detailed analysis of Centrox AI, a company that operates with a dual revenue approach, combining project-based and product-based revenue streams. This strategy allows them to maintain stability, predictability, and foster growth in the competitive software industry.\n\nCentrox AI is committed to ethical conduct, following various ethical theories such as utilitarianism, deontology, virtue ethics, and social contract theory. They emphasize integrity, professionalism, and social responsibility, aiming to create a positive work environment and maintain trust with clients and stakeholders. The company is advised'